# Exp 006: Generate EGP examples

In [39]:
import sys
sys.path.append('../source')
import api
import helpers
import pandas as pd
from collections import Counter
from dotenv import load_dotenv
import random
import os
import re
load_dotenv()
#import importlib # in case, Python module has changed
#importlib.reload(api)
SEED = int(os.getenv("SEED"))
SYSTEM_MESSAGE = {"role": "system", "content": "You are a helpful assistant."}

In [2]:
egp = helpers.get_egp()

In [134]:
construction = egp.sample(1, random_state=SEED+random.randint(0,100)).iloc[0]
construction

#                                                                 255
SuperCategory                                                 CLAUSES
SubCategory                                              subordinated
Level                                                              C2
Lexical Range                                                     NaN
guideword             FORM/USE: NON-FINITE AFTER 'ALTHOUGH', 'THOUGH'
Can-do statement    Can use non-finite clauses after subordinating...
Example             I was my parents' first and only child, and I ...
Name: 254, dtype: object

In [135]:
def get_prompt(construction, num_examples=5):
    lexical_range = ''
    if not pd.isna(construction["Lexical Range"]):
        if construction["Lexical Range"] == 1:
            lexical_range = 'low'
        elif construction["Lexical Range"] == 2:
            lexical_range = 'medium'
        elif construction["Lexical Range"] == 3:
            lexical_range = 'high'
        lexical_range = f'Use words of {lexical_range} difficulty in the rule.'
    return f'Learn the grammar rule "{construction["Can-do statement"]}" ({construction["SuperCategory"]}, {construction["SubCategory"]}, {construction["guideword"]}). It is CEFR level {construction["Level"]}. {lexical_range}\nExamples:\n{construction["Example"]}\nCreate {num_examples} more examples using that rule. Mark the words that are fulfilling it in **bold**.'
prompt = get_prompt(construction, 10)
print(prompt)

Learn the grammar rule "Can use non-finite clauses after subordinating conjunctions '(although, though)', to express contrast. " (CLAUSES, subordinated, FORM/USE: NON-FINITE AFTER 'ALTHOUGH', 'THOUGH'). It is CEFR level C2. 
Examples:
I was my parents' first and only child, and I was brought up at my grandparents' home, because, although married, my mother decided not to leave her parents' home, and my father agreed with her. 

Although committed to her job she successfully maintains her social contacts all over the world. 

Jack, though disappointed, respects the result.
Create 10 more examples using that rule. Mark the words that are fulfilling it in **bold**.


In [136]:
messages = [ SYSTEM_MESSAGE, { "role": "user", "content": prompt }]
response = api.get_openai_chat_completion(messages, max_tokens=256)[0]
print(response)

1. **Although** tired from a long day at work, she decided to go for a run.
2. He couldn't attend the party **though** invited by his friends.
3. **Although** living far away, I make sure to stay in touch with my family.
4. **Though** challenged by the difficult task, he managed to complete it on time.
5. **Although** feeling nervous, she gave a flawless performance on stage.
6. **Though** exhausted, he couldn't resist staying up late to finish the book.
7. **Although** hesitant at first, she eventually made the decision to study abroad.
8. **Though** facing financial difficulties, they embarked on their dream vacation.
9. **Although** having a busy schedule, he always finds time for his hobbies.
10. **Though** cold outside, they still decided to have a picnic in the park.


In [137]:
def get_bolds(example):
    return re.findall(r"\*\*(.*?)\*\*", example)

def parse_response(response, positive=True):
    matches = re.findall(r"^\d+\.\s+(.*)", response, re.MULTILINE)
    examples = [match for match in matches]
    bolds = [get_bolds(example) for example in examples]
    return examples, bolds

examples, bolds = parse_response(response)
print(examples)
print(bolds)

['**Although** tired from a long day at work, she decided to go for a run.', "He couldn't attend the party **though** invited by his friends.", '**Although** living far away, I make sure to stay in touch with my family.', '**Though** challenged by the difficult task, he managed to complete it on time.', '**Although** feeling nervous, she gave a flawless performance on stage.', "**Though** exhausted, he couldn't resist staying up late to finish the book.", '**Although** hesitant at first, she eventually made the decision to study abroad.', '**Though** facing financial difficulties, they embarked on their dream vacation.', '**Although** having a busy schedule, he always finds time for his hobbies.', '**Though** cold outside, they still decided to have a picnic in the park.']
[['Although'], ['though'], ['Although'], ['Though'], ['Although'], ['Though'], ['Although'], ['Though'], ['Although'], ['Though']]


In [138]:
get_bigrams = lambda words: list(zip(words, words[1:]))
get_trigrams = lambda words: list(zip(words, words[1:], words[2:]))

unigrams = [group.split(' ') for example in bolds for group in example]
bigrams = [get_bigrams(group.split(' ')) for example in bolds for group in example]
trigrams = [get_trigrams(group.split(' ')) for example in bolds for group in example]
unigram_counter = Counter([word for example in unigrams for word in example])
print(unigram_counter)
bigram_counter = Counter([" ".join(bigram) for example in bigrams for bigram in example])
print(bigram_counter)
trigram_counter = Counter([" ".join(trigram) for example in trigrams for trigram in example])
print(trigram_counter)

Counter({'Although': 5, 'Though': 4, 'though': 1})
Counter()
Counter()


Ask to write examples with this word but without using the rule.

In [139]:
def get_messages(word):
    return [ SYSTEM_MESSAGE,
            { "role": "user", "content": get_prompt(construction) },
            { "role": "assistant", "content": response },
            { "role": "user", "content": f'Is it possible to create examples using "{word}" that do NOT fulfill the rule? If so, create 5.'}]

def get_responses_for_counter(counter, top_n=2):
    for words in counter.most_common(top_n):
        if words[1] <= 1: continue
        print(get_messages(words[0])[-1])
        response = api.get_openai_chat_completion(get_messages(words[0]), model="gpt-4-0125-preview")[0]
        print(response)

for counter in [unigram_counter, bigram_counter, trigram_counter]:
    get_responses_for_counter(counter)

{'role': 'user', 'content': 'Is it possible to create examples using "Although" that do NOT fulfill the rule? If so, create 5.'}
Certainly, it's possible to use "Although" in a way that doesn't adhere to the specific rule of following it with a non-finite clause. Here are five examples where "Although" introduces full clauses instead:

1. **Although** she was tired from a long day at work, she decided to go for a run.
2. **Although** he was invited by his friends, he couldn't attend the party.
3. **Although** we live far away, I make sure to stay in touch with my family.
4. **Although** he was challenged by the difficult task, he managed to complete it on time.

{'role': 'user', 'content': 'Is it possible to create examples using "Though" that do NOT fulfill the rule? If so, create 5.'}
Yes, it's possible to create examples using "though" that do not fulfill the specific grammar rule of using non-finite clauses after subordinating conjunctions (such as "although" or "though") to expres

In [ ]:
# Maybe filter for the word tricky or push out trigrams